In [1]:
from Lib.myModules.my_ML_Modules_1 import *
import math
import sys
import contextlib

In [2]:
data_dir = "../data"
infantNum = 3
ECG_dataset = f"{data_dir}/infant{infantNum}_ecg"
RESP_dataset = f"{data_dir}/infant{infantNum}_resp"
# Read all the data
signal_ecg_0 = wfdb.rdsamp(f"{data_dir}/infant{infantNum}_ecg")
signal_resp_0 = wfdb.rdsamp(f"{data_dir}/infant{infantNum}_resp")
print(f'ECG DATA: {signal_ecg_0[1]}')
print(F'RESP DATA: {signal_resp_0[1]}')
totalDuration_ECG = num2Time(num=signal_ecg_0[1]['sig_len'], fs=signal_ecg_0[1]['fs'])
print(f'{totalDuration_ECG} sec, {secs2minutes(totalDuration_ECG)} minutes, {secs2hours(totalDuration_ECG)} hours')
totalDuration_RESP = num2Time(num=signal_resp_0[1]['sig_len'], fs=signal_resp_0[1]['fs'])
print(f'{totalDuration_RESP} sec, {secs2minutes(totalDuration_RESP)} minutes, {secs2hours(totalDuration_RESP)} hours')

ECG DATA: {'fs': 500, 'sig_len': 78684614, 'n_sig': 1, 'base_date': None, 'base_time': None, 'units': ['mV'], 'sig_name': ['II'], 'comments': []}
RESP DATA: {'fs': 50, 'sig_len': 7868296, 'n_sig': 1, 'base_date': None, 'base_time': None, 'units': ['NU'], 'sig_name': ['RESP'], 'comments': []}
157369.228 sec, 2622.8204666666666 minutes, 43.71367444444444 hours
157365.92 sec, 2622.7653333333337 minutes, 43.71275555555556 hours


In [3]:
# initialize the training
start_time_secs = hours2secs(2)
end_time_secs = hours2secs(2.2)
lr_model, pr_model, svr_rbf_model = training_pipeline(start_time_secs=start_time_secs, 
                                                        end_time_secs=end_time_secs, 
                                                        signal_ecg_0=signal_ecg_0,
                                                        signal_resp_0=signal_resp_0,
                                                        ECG_dataset=ECG_dataset,
                                                        RESP_dataset=RESP_dataset)

start: 3600000, end: 3960000
start: 360000, end: 396000
90th percentile: 0.10256002084159876, 10th percentile: -0.11833848558646011, IQR: 0.22089850642805886
Identified outliers: 17531
Data Shape: (360000,)
90th percentile: 23.560276483201907, 10th percentile: 21.39089391195727, IQR: 2.169382571244636
Identified outliers: 135
Data Shape: (36000,)
[[ 1.27967971e+02 -1.13603481e-01]
 [-1.13603481e-01  9.34161686e+01]]
Pearsons correlation: -0.001
Spearmans correlation: -0.051
---------------------------------------------
--- Linear Regression ---
Coefficient of determination: 3.6467966063602475e-06
Intercept: [123.92736]
Coefficients: [[-0.00162373]]
Mean absolute error: 5.68
Mean squared error: 95.60
Root mean squared error: 9.78
---------------------------------------------
--- Polynomial Regression ---
Coefficient of determination: 0.009791991685555557
Intercept: [122.42059]
Coefficients: [[ 1.4001151e-09  9.5668234e-08  1.2255213e-06  1.8787008e-05
  -7.5415670e-07  7.5229707e-09]]
M

In [4]:
step = 0.25
startHrs = 10
endHrs = 12
logFilePath = f"../logs/infant{infantNum}_step{step}_startHrs{startHrs}_endHrs{endHrs}.log"

with open(logFilePath, 'a') as f:
    with contextlib.redirect_stdout(f):
        for i in np.arange(startHrs, endHrs, step):
            print("-------------------------------------------------------------------------")
            print(f"Data bin index: {i:.3f}")
            start_time_secs = hours2secs(i)
            end_time_secs = hours2secs(i+step)
            lr_model, pr_model, svr_rbf_model = cascade_training_pipeline(lr_model=lr_model, 
                                                                            pr_model=pr_model, 
                                                                            svr_rbf_model=svr_rbf_model, 
                                                                            start_time_secs=start_time_secs, 
                                                                            end_time_secs=end_time_secs,
                                                                            signal_ecg_0=signal_ecg_0,
                                                                            signal_resp_0=signal_resp_0,
                                                                            ECG_dataset=ECG_dataset,
                                                                            RESP_dataset=RESP_dataset)

In [5]:
# save the model
for i in [lr_model, pr_model, svr_rbf_model]:
    filename = f"../models/infant_{infantNum}/{namestr(i, globals())[0]}_step{step}_startHrs{startHrs}_endHrs{endHrs}.sav"
    joblib.dump(i, filename)
